In [19]:
import pandas as pd

train_df = pd.read_csv('/Users/donglinxiong/Documents/i-drive/Desktop/kagglex-cohort4/train.csv')
test_df = pd.read_csv("/Users/donglinxiong/Documents/i-drive/Desktop/kagglex-cohort4/test.csv")

print(train_df.columns)
print(test_df.columns)


Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')
Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title'],
      dtype='object')


In [20]:
def get_dataset_summary(df):
    summary = {
        'Number of Rows': df.shape[0],  # number of rows
        'Number of Columns': df.shape[1],  # number of columns
        'Missing Values': df.isnull().sum().sum(),
        'Duplicates': df.duplicated().sum(),
        'Memory_usage': df.memory_usage().sum(),
        'Unique Values': df.nunique()
    }
    return summary


#get summary for both datasets
train_summary = get_dataset_summary(train_df)
test_summary = get_dataset_summary(test_df)

#Create a dataframe from the summary
summary_df = pd.DataFrame([train_summary, test_summary], index=['train', 'test'])

#print the summary table
print(summary_df)


       Number of Rows  Number of Columns  Missing Values  Duplicates  \
train           54273                 13               0           0   
test            36183                 12               0           0   

       Memory_usage                                      Unique Values  
train       5644524  id              54273
brand              53
mo...  
test        3473700  id              36183
brand              53
mo...  


In [21]:
#I want to add a new column to test dataset called 'Price' and fill it with zeros
test_df['price'] = 0


In [22]:
#Replace the Value "‚Äì" with Neighboring Value function
def replace_with_neighbor(df):
    for column in df.columns:
        #convert the column to a list for easier manipulation
        col_values = df[column].tolist()
        for i in range(len(col_values)):
            if col_values[i] == '‚Äì':
                #replace the value with the previous value
                if i > 0:
                    #if the value is not the first value in the list
                    col_values[i] = col_values[i - 1]
                    #if the value is the first value in the list
                elif i < len(col_values) - 1:
                    col_values[i] = col_values[i + 1]

        #update the column with the new values
        df[column] = col_values
    return df


#replace the value "‚Äì" in both train and test
train_df = replace_with_neighbor(train_df)
test_df = replace_with_neighbor(test_df)

#print the updated datasets
print("Train dataset:\n", train_df.head(100))
print("Test dataset:\n", test_df.head(100))



Train dataset:
     id    brand          model  model_year  milage fuel_type  \
0    0     Ford   F-150 Lariat        2018   74349  Gasoline   
1    1      BMW          335 i        2007   80000  Gasoline   
2    2   Jaguar      XF Luxury        2009   91491  Gasoline   
3    3      BMW   X7 xDrive40i        2022    2437    Hybrid   
4    4  Pontiac  Firebird Base        2001  111000  Gasoline   
..  ..      ...            ...         ...     ...       ...   
95  95  Lincoln       MKZ Base        2016   54000  Gasoline   
96  96   Nissan      Maxima SL        2007  105000  Gasoline   
97  97   Toyota     Tundra SR5        2016   74000  Gasoline   
98  98   Subaru    BRZ Limited        2016   36000  Gasoline   
99  99   Nissan    Wrangler SE        2003  106000  Gasoline   

                                               engine  \
0       375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel   
1   300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...   
2        300.0HP 4.2L 8 Cylinder Engine Gaso

In [23]:
#Function to Find Rows Containing "‚Äì" in a Dataset
def find_rows_with_value(df, value="‚Äì"):
    #finde the rows containing the specified value
    rows_with_value = df.apply(lambda row: row.astype(str).str.contains(value).any(),
                               axis=1)  #axis=1 means we are looking at rows
    return df[rows_with_value]


#find rows containing ",Äì" in the train dataset
rows_with_value_train = find_rows_with_value(train_df, value="‚Äì")
rows_with_value_test = find_rows_with_value(test_df, value="‚Äì")
print(rows_with_value_train)
print(rows_with_value_test)


Empty DataFrame
Columns: [id, brand, model, model_year, milage, fuel_type, engine, transmission, ext_col, int_col, accident, clean_title, price]
Index: []
Empty DataFrame
Columns: [id, brand, model, model_year, milage, fuel_type, engine, transmission, ext_col, int_col, accident, clean_title, price]
Index: []


In [24]:
#print all columns name in the train dataset
print(test_df.columns)

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')


In [25]:
print(train_df.columns)


Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')


In [26]:
#columns to be removed
columns_to_remove = ['fuel_type', 'ext_col', 'int_col', 'clean_title']

#remove the columns from both datasets
train_df = train_df.drop(columns=columns_to_remove)
test_df = test_df.drop(columns=columns_to_remove)

print("Updated Train dataset:\n", train_df.columns)
print("Updated Test dataset:\n", test_df.columns)

Updated Train dataset:
 Index(['id', 'brand', 'model', 'model_year', 'milage', 'engine',
       'transmission', 'accident', 'price'],
      dtype='object')
Updated Test dataset:
 Index(['id', 'brand', 'model', 'model_year', 'milage', 'engine',
       'transmission', 'accident', 'price'],
      dtype='object')


In [27]:
# identify number of unique values in each column of the train dataset and test dataset 
train_unique_counts = train_df.nunique()
test_unique_counts = test_df.nunique()

# create a dataframe from the unique counts
summary_df = pd.DataFrame({
    'train_unique_counts': train_unique_counts,
    'test_unique_counts': test_unique_counts

})

#Display the summary table
print("Summary of unique values in each column:\n", summary_df)

Summary of unique values in each column:
               train_unique_counts  test_unique_counts
id                          54273               36183
brand                          53                  53
model                        1827                1791
model_year                     34                  34
milage                       3212                2973
engine                       1061                1038
transmission                   46                  44
accident                        2                   2
price                        1481                   1


In [28]:
#compare the 'model' column between the train_df and test_df datasets, and print out the rows from train_df that have values in the 'model' column not present in the test_df.
test_models = set(test_df['model'].unique())  #get unique values in the 'model' column of test_df

#identify rows in train_df that where 'model' value are not in test_df
train_df_filtered = train_df[train_df['model'].isin(test_models)]
#unmatched_rows = train_df[~train_df['model'].isin(test_models)] #get rows in train_df where 'model' value is not in test_models

#Print the unmatched rows from train_df 
print(train_df_filtered)

          id    brand                             model  model_year  milage  \
0          0     Ford                      F-150 Lariat        2018   74349   
1          1      BMW                             335 i        2007   80000   
2          2   Jaguar                         XF Luxury        2009   91491   
3          3      BMW                      X7 xDrive40i        2022    2437   
4          4  Pontiac                     Firebird Base        2001  111000   
...      ...      ...                               ...         ...     ...   
54268  54268      BMW                      X6 xDrive50i        2017   29000   
54269  54269     Audi                   A4 2.0T Premium        2015   94634   
54270  54270  Porsche                         Cayenne S        2013   40989   
54271  54271  Porsche                 911 Carrera 4 GTS        2023    1518   
54272  54272     Audi  A5 Sportback S line Premium Plus        2021   35000   

                                                  e

In [29]:
# identify number of unique values in each column of the train dataset and test dataset 
train_unique_counts = train_df_filtered.nunique()
test_unique_counts = test_df.nunique()

# create a dataframe from the unique counts
summary_df = pd.DataFrame({
    'train_unique_counts': train_unique_counts,
    'test_unique_counts': test_unique_counts

})

#Display the summary table
print("Summary of unique values in each column:\n", summary_df)

Summary of unique values in each column:
               train_unique_counts  test_unique_counts
id                          54088               36183
brand                          52                  53
model                        1759                1791
model_year                     34                  34
milage                       3205                2973
engine                       1049                1038
transmission                   45                  44
accident                        2                   2
price                        1480                   1


In [30]:
#create sets of tuples containing the unique combinations of brand and model 
# train_combinations = set(train_df[['brand', 'model']].apply(tuple, axis=1))
# test_combinations = set(test_df[['brand', 'model']].apply(tuple, axis=1))

#identify combinations in te st_df that are not in train_df
# unmatched_combinations = test_combinations - train_combinations

#Fileter the rows in test_df with the unmatch 'brand' and 'model' combinations
# unmatched_rows = test_df[test_df[['brand', 'model']].apply(tuple, axis=1).isin(unmatched_combinations)]

#Append the unmatched rows to train_df
# train_df = pd.concat([train_df, unmatched_rows], ignore_index=True)

#Print the updated train_df
# print(train_df)

In [31]:
#identify number of unique values in each column of the train dataset and test dataset 
# /
# train_unique_counts = train_df.nunique()
# test_unique_counts = test_df.nunique()
# 
# create a dataframe from the unique counts
# summary_df = pd.DataFrame({
#      'train_unique_counts': train_unique_counts,
#      'test_unique_counts': test_unique_counts DataFrame})

# Display the summary table
# print("Summary of unique values in each column:\n", summary_df)
# /

In [32]:
#download the updated train_df to a csv file
# train_df.to_csv('/Users/donglinxiong/Documents/i-drive/Desktop/kagglex-cohort4/train_updated.csv', index=False)/

In [33]:
#download the updated test_df to a csv file
#test_df.to_csv('/Users/donglinxiong/Documents/i-drive/Desktop/kagglex-cohort4/test_updated.csv', index=False)


In [34]:
#change model_year column to mode_age current_year - model_year 
current_year = 2024
train_df['model_year'] = current_year - train_df['model_year']
test_df['model_year'] = current_year - test_df['model_year']

#print the updated datasets
print("Train dataset:\n", train_df.head())
print("Test dataset:\n", test_df.head())

Train dataset:
    id    brand          model  model_year  milage  \
0   0     Ford   F-150 Lariat           6   74349   
1   1      BMW          335 i          17   80000   
2   2   Jaguar      XF Luxury          15   91491   
3   3      BMW   X7 xDrive40i           2    2437   
4   4  Pontiac  Firebird Base          23  111000   

                                              engine  \
0      375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel   
1  300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...   
2       300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel   
3  335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...   
4      200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel   

                     transmission       accident  price  
0                    10-Speed A/T  None reported  11000  
1                     6-Speed M/T  None reported   8250  
2                     6-Speed A/T  None reported  15000  
3  Transmission w/Dual Shift Mode  None reported  63500  
4                             A/T  Non

KeyError: "None of [MultiIndex([('Acura',  1, 38170),\n            ('Acura',  2, 22753),\n            ('Acura',  3,  8252),\n            ('Acura',  3, 33835),\n            ('Acura',  3, 34049),\n            ('Acura',  4,  4911),\n            ('Acura',  4,  9762),\n            ('Acura',  4, 48232),\n            ('Acura',  5,  8219),\n            ('Acura',  5, 24406),\n            ...\n            ('Volvo', 13, 18261),\n            ('Volvo', 13, 34704),\n            ('Volvo', 14, 51796),\n            ('Volvo', 15, 21358),\n            ('Volvo', 16,  2954),\n            ('Volvo', 16,  6928),\n            ('Volvo', 19, 39376),\n            ('Volvo', 19, 51593),\n            ('Volvo', 20, 33765),\n            ('Volvo', 29, 28930)],\n           names=['brand', 'model_year', None], length=3443)] are in the [index]"